In [5]:
# pre-requisites 

# ! pip install python-dotenv > /dev/null
# print("Installed python-dotenv")
# ! pip install transformers torch huggingface_hub torchvision datasets > /dev/null
# print("Installed transformers, torch, huggingface_hub, datasets and torchvision")
# ! pip install accelerate > /dev/null 
# print("Installed accelerate")

# Imports 
import pandas as pd 
import torch
import os 
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset 
from huggingface_hub import login
import training_script

#ENV Variables 
TOKEN = os.getenv("TOKEN")
TOKENIZERS_PARALLELISM = os.getenv("TOKENIZERS_PARALLELISM")
DATASET_PATH = 'fol_sympy_nl_16k.csv' 
TEST_SIZE = 0.2
SEED = 42

MODEL_LIST = [
    "google/gemma-3-1b-it"
]
login(token = TOKEN)

print("Initialised environment variables")

OSError: g is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

Writing the predefinded functions in the cell below

In [2]:
def preprocess(batch):
     inputs = tokenizer(
         batch['natural_language'],
         max_length=128,
         truncation=True,
         padding = 'max_length',
         
     )    
    
     labels = tokenizer(
         batch['sympy'],
         max_length =128,
         truncation = True, 
         padding = 'max_length'
     )
     inputs['labels']=labels['input_ids']
     return inputs

In [5]:
# Running the training loop in order to train the model 

for model in MODEL_LIST:
    
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForCausalLM.from_pretrained(model)
    
    dataset = load_dataset('csv', data_files = DATASET_PATH )['train']
    split_dataset = dataset.train_test_split(test_size = TEST_SIZE, seed = SEED)
    
    train_set = split_dataset['train']
    test_set = split_dataset['test']

    train_tokenized = train_set.map(preprocess, batched=True)
    test_tokenized = test_set.map(preprocess, batched=True)
    
    training_args = TrainingArguments(
        
            output_dir = "./results",
            per_device_train_batch_size = 12,
            per_device_eval_batch_size = 12,
            eval_strategy = 'epoch',
            save_strategy = 'epoch',
            do_eval = True,
            logging_steps = 2,
            eval_steps = 2, 
            num_train_epochs = 1,
            learning_rate = 2e-5,
    
    )
    
    trainer = Trainer(
        
            model = model,
            args = training_args,
            train_dataset = train_tokenized,
            eval_dataset = test_tokenized,
    
    )

    trainer.train()
    
    # Post training 
    
    # model.save_pretrained("/usr3/allanp/llm_reasoning_notebook_experiments/model_ckpts", from_pt=True) 
    
    # Model evaluation 
    
    # Clear data usage 
    del model, tokenizer
    torch.cuda.synchronize()  # Soft stop the GPU and ensure all processes finish
    torch.cuda.empty_cache()  # Clear GPU RAM before starting the next execution
    
    
    

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:

# df=pd.read_csv("fol_dataset_reversed.csv")
# column_titles=['english_description','expression']
# df=df.reindex(columns=column_titles)
# df.to_csv('fol_dataset.csv', index = False)

In [ ]:
import json

logs = trainer.state.log_history

      